In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import time
import re
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [20]:
chrome_driver_path = '/opt/homebrew/Caskroom/chromedriver/100.0.4896.60/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument("headless")

In [10]:
# 스크롤 다운
def scroll_down(second):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(second)
    new = driver.execute_script("return document.documentElement.scrollHeight")
    
    return new

# 한국 TV 프로그램 인기 순위

In [21]:
driver = webdriver.Chrome(chrome_driver_path, options=options)
pedia = 'https://pedia.watcha.com/ko-KR/?domain=tv'
driver.get(pedia)
time.sleep(1)

#한국 TV프로그램 인기순위 리스트 갖고오기
#팝업닫기
driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div/div/button').click()
#next버튼
for i in range(5):
    driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/section/div/section/div[1]/div[2]/div/div[5]/div/img').click()
    time.sleep(2)

dramas_title = []
dramas_url = []
for i in range(1,31):
    li = driver.find_element_by_xpath(f'/html/body/div/div/div[1]/section/div/section/div[1]/div[2]/div/div[1]/div/div/ul/li[{i}]/a')
    title = li.get_attribute('title')
    href = li.get_attribute('href')
    dramas_title.append(title)
    dramas_url.append(href)

#작품별로 이동하여 댓글 수집
content_dic = {'title' : [], 'review' : []}
for i, url in enumerate(dramas_url):
    driver.get(url + str('/comments'))
    time.sleep(2)
    
    #팝업닫기
    driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div/div/button').click()
    
    #스크롤 끝까지 내리기
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        new_height = scroll_down(2)
        if new_height == last_height:
            new_height = scroll_down(2)
            if new_height == last_height:
                break
        last_height = new_height
    time.sleep(2)
    
    # 댓글 가져오기
    for j in range(1,8000):
        try:
            review = driver.find_element_by_xpath(f'/html/body/div[1]/div/div[1]/section/section/div/div/div/ul/div[{j}]/div[2]').text
            if review == '스포일러가 있어요!! 보기':
                rev_url = driver.find_element_by_xpath(f'/html/body/div/div/div[1]/section/section/div/div/div/ul/div[{j}]/div[2]/a').get_attribute('href')
                # 새탭열기
                driver.execute_script('window.open("");')
                # 새탭으로 스위치
                driver.switch_to.window(driver.window_handles[-1])
                driver.get(rev_url)
                time.sleep(1)

                driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div/div/button').click()
                driver.find_element_by_xpath('/html/body/div/div/div[1]/section/div/section/section/div/div/section[1]/div[2]/span/button').click()

                review = driver.find_element_by_xpath('/html/body/div/div/div[1]/section/div/section/section/div/div/section[1]/div[2]/div/span').text
                content_dic['title'].append(dramas_title[i])
                content_dic['review'].append(review)
                driver.close()

                driver.switch_to.window(driver.window_handles[-1])
            else:
                content_dic['title'].append(dramas_title[i])
                content_dic['review'].append(review)
        except:
            break
    print(f"{i+1}번째 작품 done")

1번째 작품 done
2번째 작품 done
3번째 작품 done
4번째 작품 done
5번째 작품 done
6번째 작품 done
7번째 작품 done
8번째 작품 done
9번째 작품 done
10번째 작품 done
11번째 작품 done
12번째 작품 done
13번째 작품 done
14번째 작품 done
15번째 작품 done
16번째 작품 done
17번째 작품 done
18번째 작품 done
19번째 작품 done
20번째 작품 done
21번째 작품 done
22번째 작품 done
23번째 작품 done
24번째 작품 done
25번째 작품 done
26번째 작품 done
27번째 작품 done
28번째 작품 done
29번째 작품 done
30번째 작품 done


In [22]:
df_content_dic = pd.DataFrame(content_dic)

title     0
review    0
dtype: int64

In [23]:
df_content_dic.to_csv("df_contents.csv")

In [24]:
len(df_content_dic)

25676